In [79]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

base_path = 'C:/Users/mskac/machineLearning/GasLeakConEd/data/'

In [80]:
# create a list of dataframes containing ecoding of the column names
mypath_headers = base_path+ 'original/acs_data/acs_dp05/column_names/'
head_dfs = []
head_files = [f for f in listdir(mypath_headers) if isfile(join(mypath_headers, f))]

for head_file in head_files:
    head_dfs.append(pd.read_csv(join(mypath_headers, head_file)))

In [81]:
#create a list of dataframes with data for each year

dfs = []

mypath_from = base_path+ 'original/acs_data/acs_dp05/data_files/'
mypath_to = base_path+ 'processed/important_(used_in_app)/'

files = [f for f in listdir(mypath_from) if isfile(join(mypath_from, f))]

for file in files:
    dfs.append(pd.read_csv(join(mypath_from, file)))

In [82]:
# clean from punctuation

d = {'[': '', ']': '', '"': ''}
for df in dfs:
    for old, new in d.items():
        df.columns = df.columns.str.replace(old, new)
        for column in df.columns:
            df[column] = df[column].astype(str).str.replace(old, new)

In [83]:
# there is an extra column in the end, so drop it
for i in range(len(dfs)):
    dfs[i] = dfs[i].drop(dfs[i].columns[-1],axis=1)

In [84]:
# rename columns from codes to meaningful names

# create a list of dictionaries for codes meanings for each year
names = []
for df_head in head_dfs:
    names.append(df_head.to_dict())
    
# for each year compare codes in the dictionary and codes in datafiles. If a code from file is found in a dict - replace with 
# a meaningful name, if not - replace with "unknown"
for index in range (len(dfs)):      
    new_columns = [names[index][column][0] if column in names[index] else "unknown" for column in dfs[index].columns[:-3]]
    new_columns.extend(['state', 'county', 'tract'])
    dfs[index].columns = new_columns

In [85]:
for df in dfs:
    df.drop(['unknown'], axis = 1, inplace = True)

In [86]:
# run this to print all the columns/rows without truncating
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [87]:
# columns have different names in different years. If we remove !!Total population from each name, then names will match

for i in range(len(dfs)):
    dfs[i].columns=dfs[i].columns.str.replace('!!Total population','')

In [88]:
dfs[0].columns.to_list()

['id',
 'Estimate!!SEX AND AGE',
 'Estimate Margin of Error!!SEX AND AGE',
 'Percent!!SEX AND AGE',
 'Percent Margin of Error!!SEX AND AGE',
 'Estimate!!SEX AND AGE!!Male',
 'Estimate Margin of Error!!SEX AND AGE!!Male',
 'Percent!!SEX AND AGE!!Male',
 'Percent Margin of Error!!SEX AND AGE!!Male',
 'Estimate!!SEX AND AGE!!Female',
 'Estimate Margin of Error!!SEX AND AGE!!Female',
 'Percent!!SEX AND AGE!!Female',
 'Percent Margin of Error!!SEX AND AGE!!Female',
 'Estimate!!SEX AND AGE!!Under 5 years',
 'Estimate Margin of Error!!SEX AND AGE!!Under 5 years',
 'Percent!!SEX AND AGE!!Under 5 years',
 'Percent Margin of Error!!SEX AND AGE!!Under 5 years',
 'Estimate!!SEX AND AGE!!5 to 9 years',
 'Estimate Margin of Error!!SEX AND AGE!!5 to 9 years',
 'Percent!!SEX AND AGE!!5 to 9 years',
 'Percent Margin of Error!!SEX AND AGE!!5 to 9 years',
 'Estimate!!SEX AND AGE!!10 to 14 years',
 'Estimate Margin of Error!!SEX AND AGE!!10 to 14 years',
 'Percent!!SEX AND AGE!!10 to 14 years',
 'Percent 

In [89]:
# we need to use absolute values for aggregation purposes, so drop unnecessary rows
for i in range(len(dfs)):
    dfs_columns = dfs[i].columns
    columns = [column for column in dfs_columns if 'Percent' not in column and 'Margin of Error' not in column and 'Margin of Error' not in column]
    dfs[i] = dfs[i][columns]

In [90]:
for i in range(len(dfs)):
    dfs_columns = dfs[i].columns
    columns = [column for column in dfs_columns if 'RACE' in column]
    columns.extend([
 'state',
 'county',
 'tract'])
    dfs[i] = dfs[i][columns]
dfs[1].columns.to_list()

['Estimate!!RACE',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!Two or more races',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race',
 'Estimate!!RACE!!One race!!White',
 'Estimate!!RACE!!One race!!Black or African American',
 'Estimate!!RACE!!One race!!American Indian and Alaska Native',
 'Estimate!!RACE!!One race!!American Indian and Alaska Na

In [91]:
short_columns = ['Estimate!!RACE!!One race!!White',
 'Estimate!!RACE!!One race!!Black or African American',
 'Estimate!!RACE!!One race!!American Indian and Alaska Native',
'Estimate!!RACE!!One race!!Asian',
'Estimate!!RACE!!One race!!Native Hawaiian and Other Pacific Islander',
'Estimate!!RACE!!Two or more races!!White and Black or African American',
 'Estimate!!RACE!!Two or more races!!White and American Indian and Alaska Native',
 'Estimate!!RACE!!Two or more races!!White and Asian',
 'Estimate!!RACE!!Two or more races!!Black or African American and American Indian and Alaska Native',
'Estimate!!RACE!!One race!!Some other race',
'Estimate!!HISPANIC OR LATINO AND RACE!!Hispanic or Latino (of any race)',
                 
 'Estimate!!RACE!!One race!!American Indian and Alaska Native!!Cherokee tribal grouping',
 'Estimate!!RACE!!One race!!American Indian and Alaska Native!!Chippewa tribal grouping',
 'Estimate!!RACE!!One race!!American Indian and Alaska Native!!Navajo tribal grouping',
 'Estimate!!RACE!!One race!!American Indian and Alaska Native!!Sioux tribal grouping',
 
 'Estimate!!RACE!!One race!!Asian!!Asian Indian',
 'Estimate!!RACE!!One race!!Asian!!Chinese',
 'Estimate!!RACE!!One race!!Asian!!Filipino',
 'Estimate!!RACE!!One race!!Asian!!Japanese',
 'Estimate!!RACE!!One race!!Asian!!Korean',
 'Estimate!!RACE!!One race!!Asian!!Vietnamese',
 'Estimate!!RACE!!One race!!Asian!!Other Asian',
                 
 'Estimate!!HISPANIC OR LATINO AND RACE!!Hispanic or Latino (of any race)!!Mexican',
 'Estimate!!HISPANIC OR LATINO AND RACE!!Hispanic or Latino (of any race)!!Puerto Rican',
 'Estimate!!HISPANIC OR LATINO AND RACE!!Hispanic or Latino (of any race)!!Cuban',
 'Estimate!!HISPANIC OR LATINO AND RACE!!Hispanic or Latino (of any race)!!Other Hispanic or Latino',
 'state',
 'county',
 'tract'
 ]

In [92]:
for i in range(len(dfs)):
    dfs[i] = dfs[i][short_columns]

In [93]:
for i in range(len(dfs)):
    dfs[i].columns=dfs[i].columns.str.replace('Estimate!!RACE!!One race!!','')
    dfs[i].columns=dfs[i].columns.str.replace('Estimate!!RACE!!Two or more races!!','')
    dfs[i].columns = dfs[i].columns.str.replace('Estimate!!HISPANIC OR LATINO AND RACE!!', '')
    dfs[i].columns = dfs[i].columns.str.replace('!!',':')
dfs[0].columns.to_list()

['White',
 'Black or African American',
 'American Indian and Alaska Native',
 'Asian',
 'Native Hawaiian and Other Pacific Islander',
 'White and Black or African American',
 'White and American Indian and Alaska Native',
 'White and Asian',
 'Black or African American and American Indian and Alaska Native',
 'Some other race',
 'Hispanic or Latino (of any race)',
 'American Indian and Alaska Native:Cherokee tribal grouping',
 'American Indian and Alaska Native:Chippewa tribal grouping',
 'American Indian and Alaska Native:Navajo tribal grouping',
 'American Indian and Alaska Native:Sioux tribal grouping',
 'Asian:Asian Indian',
 'Asian:Chinese',
 'Asian:Filipino',
 'Asian:Japanese',
 'Asian:Korean',
 'Asian:Vietnamese',
 'Asian:Other Asian',
 'Hispanic or Latino (of any race):Mexican',
 'Hispanic or Latino (of any race):Puerto Rican',
 'Hispanic or Latino (of any race):Cuban',
 'Hispanic or Latino (of any race):Other Hispanic or Latino',
 'state',
 'county',
 'tract']

In [94]:
for i in range(len(dfs)):
    for column in dfs[i]:
        dfs[i][column] = dfs[i][column].astype('float')

In [95]:
for i in range(len(dfs)):
    dfs[i]['total_population'] = dfs[i].loc[:, 'White':'Some other race'].sum(axis=1)
dfs[i].head(10)    

,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,White and Black or African American,White and American Indian and Alaska Native,White and Asian,Black or African American and American Indian and Alaska Native,Some other race,Hispanic or Latino (of any race),American Indian and Alaska Native:Cherokee tribal grouping,American Indian and Alaska Native:Chippewa tribal grouping,American Indian and Alaska Native:Navajo tribal grouping,American Indian and Alaska Native:Sioux tribal grouping,Asian:Asian Indian,Asian:Chinese,Asian:Filipino,Asian:Japanese,Asian:Korean,Asian:Vietnamese,Asian:Other Asian,Hispanic or Latino (of any race):Mexican,Hispanic or Latino (of any race):Puerto Rican,Hispanic or Latino (of any race):Cuban,Hispanic or Latino (of any race):Other Hispanic or Latino,state,county,tract,total_population
0,2830.0,37.0,0.0,45.0,0.0,59.0,24.0,29.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,31.0,0.0,6.0,0.0,0.0,46.0,36.0,91.0,61404.0,3024.0
1,3407.0,39.0,8.0,56.0,0.0,0.0,45.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,1.0,33.0,5.0,0.0,3.0,15.0,0.0,7.0,36.0,95.0,740500.0,3555.0
2,2023.0,6.0,3.0,7.0,0.0,7.0,25.0,2.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,1.0,10.0,7.0,6.0,36.0,89.0,492900.0,2074.0
3,3875.0,232.0,16.0,11.0,7.0,43.0,15.0,59.0,5.0,83.0,169.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,23.0,73.0,0.0,73.0,36.0,89.0,492100.0,4346.0
4,6218.0,192.0,0.0,302.0,0.0,47.0,27.0,70.0,0.0,12.0,133.0,0.0,0.0,0.0,0.0,224.0,34.0,0.0,0.0,44.0,0.0,0.0,12.0,20.0,0.0,101.0,36.0,91.0,62405.0,6868.0
5,6130.0,549.0,18.0,28.0,0.0,28.0,0.0,75.0,4.0,189.0,345.0,5.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,68.0,158.0,0.0,119.0,36.0,89.0,492700.0,7021.0
6,2525.0,25.0,27.0,0.0,0.0,97.0,20.0,0.0,0.0,4.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,31.0,10.0,48.0,36.0,91.0,60601.0,2698.0
7,4976.0,9.0,0.0,8.0,0.0,0.0,0.0,24.0,0.0,43.0,21.0,0.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,12.0,9.0,0.0,0.0,36.0,91.0,60602.0,5060.0
8,3038.0,174.0,7.0,221.0,0.0,33.0,10.0,23.0,0.0,178.0,498.0,0.0,0.0,0.0,0.0,203.0,0.0,14.0,0.0,0.0,0.0,4.0,0.0,277.0,0.0,221.0,36.0,93.0,33200.0,3684.0
9,2267.0,386.0,0.0,185.0,0.0,49.0,6.0,72.0,0.0,42.0,233.0,0.0,0.0,0.0,0.0,48.0,94.0,32.0,0.0,3.0,8.0,0.0,9.0,75.0,0.0,149.0,36.0,93.0,33400.0,3007.0


In [96]:
# to get geoid we need to make sure state, county and tract columns have lengths of 2, 3, and 6 accordingly, 
# so pad with zeros on the left

for i in range(len(dfs)):
    dfs[i]['state'] = dfs[i]['state'].astype(int).astype(str).str.zfill(2)
    dfs[i]['county'] = dfs[i]['county'].astype(int).astype(str).str.zfill(3)
    dfs[i]['tract'] = dfs[i]['tract'].astype(int).astype(str).str.zfill(6)
    dfs[i]['geoid'] = dfs[i].state.astype(str)+dfs[i].county.astype(str)+dfs[i].tract.astype(str)
    dfs[i] = dfs[i].drop({'state', 'county', 'tract'}, axis = 1)

In [97]:
dfs[0].head()

,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,White and Black or African American,White and American Indian and Alaska Native,White and Asian,Black or African American and American Indian and Alaska Native,Some other race,Hispanic or Latino (of any race),American Indian and Alaska Native:Cherokee tribal grouping,American Indian and Alaska Native:Chippewa tribal grouping,American Indian and Alaska Native:Navajo tribal grouping,American Indian and Alaska Native:Sioux tribal grouping,Asian:Asian Indian,Asian:Chinese,Asian:Filipino,Asian:Japanese,Asian:Korean,Asian:Vietnamese,Asian:Other Asian,Hispanic or Latino (of any race):Mexican,Hispanic or Latino (of any race):Puerto Rican,Hispanic or Latino (of any race):Cuban,Hispanic or Latino (of any race):Other Hispanic or Latino,total_population,geoid
0,111.0,2249.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,208.0,397.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,234.0,15.0,148.0,2587.0,36005039000
1,1427.0,1219.0,0.0,134.0,0.0,16.0,0.0,0.0,0.0,4200.0,5395.0,0.0,0.0,0.0,0.0,26.0,16.0,81.0,11.0,0.0,0.0,0.0,2601.0,1130.0,68.0,1596.0,6996.0,36005039100
2,90.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,0.0,67.0,1761.0,36005039200
3,1120.0,2211.0,24.0,292.0,0.0,201.0,0.0,0.0,0.0,3928.0,5134.0,0.0,0.0,0.0,0.0,0.0,44.0,75.0,0.0,0.0,173.0,0.0,423.0,2915.0,63.0,1733.0,7776.0,36005039300
4,295.0,3121.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,831.0,1232.0,0.0,0.0,0.0,0.0,10.0,19.0,0.0,0.0,0.0,0.0,0.0,66.0,530.0,0.0,636.0,4276.0,36005039400


In [98]:
# make one big df with incident year column
for i in range(len(dfs)):
    dfs[i]['incident_year'] = 2010 + i


In [99]:
all_years_df = pd.concat(dfs, ignore_index = True)
all_years_df['geoid'] = all_years_df['geoid'].astype('int64')

In [100]:
all_years_df.columns.to_list()

['White',
 'Black or African American',
 'American Indian and Alaska Native',
 'Asian',
 'Native Hawaiian and Other Pacific Islander',
 'White and Black or African American',
 'White and American Indian and Alaska Native',
 'White and Asian',
 'Black or African American and American Indian and Alaska Native',
 'Some other race',
 'Hispanic or Latino (of any race)',
 'American Indian and Alaska Native:Cherokee tribal grouping',
 'American Indian and Alaska Native:Chippewa tribal grouping',
 'American Indian and Alaska Native:Navajo tribal grouping',
 'American Indian and Alaska Native:Sioux tribal grouping',
 'Asian:Asian Indian',
 'Asian:Chinese',
 'Asian:Filipino',
 'Asian:Japanese',
 'Asian:Korean',
 'Asian:Vietnamese',
 'Asian:Other Asian',
 'Hispanic or Latino (of any race):Mexican',
 'Hispanic or Latino (of any race):Puerto Rican',
 'Hispanic or Latino (of any race):Cuban',
 'Hispanic or Latino (of any race):Other Hispanic or Latino',
 'total_population',
 'geoid',
 'incident_year

In [101]:
all_years_df['Two races'] = all_years_df['White and Black or African American'] + \
                            all_years_df['White and American Indian and Alaska Native']+ \
                            all_years_df['White and Asian'] +\
                            all_years_df['Black or African American and American Indian and Alaska Native']

In [102]:
simplified_columns = ['White',
 'Black or African American',
 'American Indian and Alaska Native',
 'Asian',
 'Native Hawaiian and Other Pacific Islander',
 'Some other race',
 'Hispanic or Latino (of any race)',
 'Asian:Asian Indian',
 'Asian:Chinese',
 'Asian:Filipino',
 'Asian:Japanese',
 'Asian:Korean',
 'Asian:Vietnamese',
 'Asian:Other Asian',
 'Hispanic or Latino (of any race):Mexican',
 'Hispanic or Latino (of any race):Puerto Rican',
 'Hispanic or Latino (of any race):Cuban',
 'Hispanic or Latino (of any race):Other Hispanic or Latino',
 'total_population',
 'geoid',
 'incident_year',
 'Two races']

In [103]:
all_years_df = all_years_df[simplified_columns]

In [104]:
all_years_df.rename(columns={'Black or African American':'Black | Afr American',
                             'American Indian and Alaska Native':'American Indian|Alaska Native',
                            'Native Hawaiian and Other Pacific Islander':'Native Hawaiian',
                            'Some other race':'Other race',
                            'Hispanic or Latino (of any race)':'Hispanic or Latino (any race)',
                            'Hispanic or Latino (of any race):Mexican':'Hispanic or Latino: Mexican',
                            'Hispanic or Latino (of any race):Puerto Rican':'Hispanic or Latino: Puerto Rican',
                            'Hispanic or Latino (of any race):Cuban':'Hispanic or Latino: Cuban'
                            }, inplace = True)

In [105]:
all_years_df.drop('Hispanic or Latino (of any race):Other Hispanic or Latino', axis = 1, inplace = True)

In [106]:
all_years_df.columns = all_years_df.columns.str.replace(" or ", "|")
all_years_df.columns

Index(['White', 'Black | Afr American', 'American Indian|Alaska Native',
       'Asian', 'Native Hawaiian', 'Other race', 'Hispanic|Latino (any race)',
       'Asian:Asian Indian', 'Asian:Chinese', 'Asian:Filipino',
       'Asian:Japanese', 'Asian:Korean', 'Asian:Vietnamese',
       'Asian:Other Asian', 'Hispanic|Latino: Mexican',
       'Hispanic|Latino: Puerto Rican', 'Hispanic|Latino: Cuban',
       'total_population', 'geoid', 'incident_year', 'Two races'],
      dtype='object')

In [107]:
df_gas = pd.read_csv(base_path+ 'processed/FULL_fdny_2013_2020.csv')
df_gas['geoid'] = df_gas['geoid'].astype('int64')
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['count'] = 1
df_gas_.head()


,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,count
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,1
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,1
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,1
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,1
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,1


In [108]:
df_gas_ = df_gas_.groupby(['incident_year', 'geoid']).agg({'count': 'count'}).reset_index()
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
all_years_df['incident_year'] = all_years_df['incident_year'].astype('int64')
merged_dfs = all_years_df.merge(df_gas_, how='inner', on = ['incident_year','geoid'], validate="one_to_one")
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'geoid'])
merged_dfs.columns.to_list()

['White',
 'Black | Afr American',
 'American Indian|Alaska Native',
 'Asian',
 'Native Hawaiian',
 'Other race',
 'Hispanic|Latino (any race)',
 'Asian:Asian Indian',
 'Asian:Chinese',
 'Asian:Filipino',
 'Asian:Japanese',
 'Asian:Korean',
 'Asian:Vietnamese',
 'Asian:Other Asian',
 'Hispanic|Latino: Mexican',
 'Hispanic|Latino: Puerto Rican',
 'Hispanic|Latino: Cuban',
 'total_population',
 'geoid',
 'incident_year',
 'Two races',
 'count']

In [109]:
merged_dfs['gas_leaks'] = merged_dfs['count']
merged_dfs['gas_leaks_per_person'] = merged_dfs['gas_leaks']/merged_dfs['total_population']

In [110]:
merged_dfs.drop(columns={'count'}, inplace = True)

In [111]:
merged_dfs.to_csv(base_path+ 'processed/important_(used_in_app)/nationalities_data.csv')

In [112]:
nation_all = merged_dfs.groupby(['geoid']).agg('sum').reset_index()
nation_all['gas_leaks_per_person'] = nation_all['gas_leaks']/nation_all['total_population']

In [113]:
nation_all.head()

,geoid,White,Black | Afr American,American Indian|Alaska Native,Asian,Native Hawaiian,Other race,Hispanic|Latino (any race),Asian:Asian Indian,Asian:Chinese,Asian:Filipino,Asian:Japanese,Asian:Korean,Asian:Vietnamese,Asian:Other Asian,Hispanic|Latino: Mexican,Hispanic|Latino: Puerto Rican,Hispanic|Latino: Cuban,total_population,incident_year,Two races,gas_leaks,gas_leaks_per_person
0,36005000200,12373.0,7868.0,51.0,1225.0,0.0,7467.0,22067.0,152.0,0.0,0.0,67.0,341.0,0.0,665.0,1117.0,11535.0,142.0,30061.0,12093,1077.0,38,0.001264
1,36005000400,12926.0,11356.0,109.0,527.0,0.0,8663.0,22427.0,174.0,183.0,124.0,0.0,9.0,37.0,0.0,423.0,14239.0,559.0,34056.0,12093,475.0,49,0.001439
2,36005001600,13395.0,12661.0,0.0,24.0,0.0,7860.0,22732.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,356.0,11263.0,459.0,34158.0,12093,218.0,99,0.002898
3,36005001900,3873.0,5447.0,0.0,349.0,0.0,5666.0,8878.0,164.0,39.0,15.0,70.0,47.0,0.0,14.0,1612.0,3175.0,33.0,15487.0,12093,152.0,38,0.002454
4,36005002000,15636.0,23951.0,1060.0,1141.0,0.0,10602.0,33119.0,628.0,0.0,0.0,0.0,0.0,419.0,94.0,2088.0,18786.0,175.0,52625.0,12093,235.0,314,0.005967


In [116]:
nation_all.to_csv(base_path+ 'processed/important_(used_in_app)/nationalities_data_all.csv')

In [124]:
# create a list of dataframes containing ecoding of the column names
mypath_headers = base_path+ 'original/acs_data/acs_dp04/column_names/'
head_dfs = []
head_files = [f for f in listdir(mypath_headers) if isfile(join(mypath_headers, f))]

for head_file in head_files:
    head_dfs.append(pd.read_csv(join(mypath_headers, head_file)))

In [125]:
#create a list of dataframes with data for each year

dfs = []

mypath_from = base_path+ 'original/acs_data/acs_dp04/data_files/'
mypath_to =base_path+ 'processed/acs_data/acs_dp04/'

files = [f for f in listdir(mypath_from) if isfile(join(mypath_from, f))]

for file in files:
    dfs.append(pd.read_csv(join(mypath_from, file),low_memory=False))

In [126]:
d = {'[': '', ']': '', '"': ''}
for df in dfs:
    for old, new in d.items():
        df.columns = df.columns.str.replace(old, new)
        for column in df.columns:
            df[column] = df[column].astype(str).str.replace(old, new)

In [127]:
for i in range(len(dfs)):
    cols = [c for c in dfs[i].columns if c[:7] != 'Unnamed']
    dfs[i]=dfs[i][cols]

In [128]:
# create a list of dictionaries for codes meanings for each year

names = []
for df_head in head_dfs:
    names.append(df_head.to_dict())

In [129]:
# for each year compare codes in the dictionary and codes in datafiles. If a code from file is found in a dict - replace with 
# a meaningful name, if not - replace with "unknown"
for index in range (len(dfs)):      
    new_columns = [names[index][column][0] if column in names[index] else "unknown" for column in dfs[index].columns[:-3]]
    new_columns.extend(['state', 'county', 'tract'])
    dfs[index].columns = new_columns


In [130]:
print(dfs[0].columns)

Index(['id', 'Estimate!!HOUSING OCCUPANCY!!Total housing units',
       'Estimate Margin of Error!!HOUSING OCCUPANCY!!Total housing units',
       'Percent!!HOUSING OCCUPANCY!!Total housing units',
       'Percent Margin of Error!!HOUSING OCCUPANCY!!Total housing units',
       'Estimate!!HOUSING OCCUPANCY!!Occupied housing units',
       'Estimate Margin of Error!!HOUSING OCCUPANCY!!Occupied housing units',
       'Percent!!HOUSING OCCUPANCY!!Occupied housing units',
       'Percent Margin of Error!!HOUSING OCCUPANCY!!Occupied housing units',
       'Estimate!!HOUSING OCCUPANCY!!Vacant housing units',
       ...
       'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown',
       'unknown', 'state', 'county', 'tract'],
      dtype='object', length=1133)


In [131]:
for df in dfs:
    df.drop(['unknown'], axis = 1, inplace = True)

In [132]:
for i in range(len(dfs)):
    columns = [column for column in dfs[i] if 'heating' in column.lower() and 'Estimate!!' in column]
    columns.extend(['state', 'county', 'tract'])
    dfs[i] = dfs[i][columns]


In [133]:
for i in range(len(dfs)):
    dfs[i].columns=dfs[i].columns.str.replace('Estimate!!HOUSE HEATING FUEL!!','')
    dfs[i].columns=dfs[i].columns.str.replace('Occupied housing units','')
    dfs[i].columns=dfs[i].columns.str.replace('!!','')
    dfs[i] = dfs[i].drop(columns={''}, axis = 1)
dfs[0].head()

,Utility gas,"Bottled, tank, or LP gas",Electricity,"Fuel oil, kerosene, etc.",Coal or coke,Wood,Solar energy,Other fuel,No fuel used,state,county,tract
0,314,0,12,481,0,0,0,12,0,36,5,039000
1,345,13,78,1678,0,0,0,10,0,36,5,039100
2,106,0,18,482,0,0,0,0,0,36,5,039200
3,597,14,410,2011,47,0,0,61,64,36,5,039300
4,474,34,198,846,0,0,0,11,10,36,5,039400


In [134]:
for i in range(len(dfs)):
    for column in dfs[i]:
        dfs[i][column] = dfs[i][column].astype('float')

In [135]:
# to get geoid we need to make sure state, county and tract columns have lengths of 2, 3, and 6 accordingly, 
# so pad with zeros on the left

for i in range(len(dfs)):
    dfs[i]['state'] = dfs[i]['state'].astype(int).astype(str).str.zfill(2)
    dfs[i]['county'] = dfs[i]['county'].astype(int).astype(str).str.zfill(3)
    dfs[i]['tract'] = dfs[i]['tract'].astype(int).astype(str).str.zfill(6)
    dfs[i]['geoid'] = dfs[i].state.astype(str)+dfs[i].county.astype(str)+dfs[i].tract.astype(str)
    dfs[i] = dfs[i].drop({'state', 'county', 'tract'}, axis = 1)

In [136]:
# make one big df with incident year column
for i in range(len(dfs)):
    dfs[i]['incident_year'] = 2010 + i


In [137]:
all_years_df = pd.concat(dfs, ignore_index = True)
all_years_df['geoid'] = all_years_df['geoid'].astype('int64')

In [138]:
df_gas = pd.read_csv(base_path+ 'processed/FULL_fdny_2013_2020.csv')
df_gas['geoid'] = df_gas['geoid'].astype('int64')
df_gas_ = df_gas
df_gas_['incident_year'] = df_gas['incident_date_time'].str[6:10]
df_gas_['incident_month'] = df_gas['incident_date_time'].str[:2]
df_gas_['count'] = 1
df_gas_.head()


,Unnamed: 0,geoid,fire_box,ntaname,boroname,incident_date_time,engines_assigned_quantity,total_incident_duration,incident_year,incident_month,count
0,0,36047040000,B4235,Bensonhurst East,Brooklyn,01/06/2013 07:05:52 PM,2,1035,2013,01,1
1,1,36047069602,B3183,Georgetown-Marine Park-Bergen Beach-Mill Basin,Brooklyn,01/13/2013 07:21:51 AM,3,1469,2013,01,1
2,2,36047052000,B2475,Flatbush,Brooklyn,07/30/2017 04:40:45 PM,2,481,2017,07,1
3,3,36047035300,B1012,Crown Heights North,Brooklyn,02/24/2020 10:11:19 AM,2,558,2020,02,1
4,4,36047007100,B0582,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,Brooklyn,12/19/2019 10:18:32 AM,2,1158,2019,12,1


In [139]:
df_gas_ = df_gas_.groupby(['incident_year', 'geoid']).agg({'count': 'count'}).reset_index()
df_gas_['incident_year'] = df_gas_['incident_year'].astype('int64')
all_years_df['incident_year'] = all_years_df['incident_year'].astype('int64')
merged_dfs = all_years_df.merge(df_gas_, how='inner', on = ['incident_year','geoid'], validate="one_to_one")
merged_dfs = merged_dfs.sort_values(by = ['incident_year', 'geoid'])
merged_dfs.columns.to_list()

['Utility gas',
 'Bottled, tank, or LP gas',
 'Electricity',
 'Fuel oil, kerosene, etc.',
 'Coal or coke',
 'Wood',
 'Solar energy',
 'Other fuel',
 'No fuel used',
 'geoid',
 'incident_year',
 'Percent ',
 'Percent Utility gas',
 'Bottled tank or LP gas',
 'Percent Bottled tank or LP gas',
 'Percent Electricity',
 'Fuel oil kerosene etc.',
 'Percent Fuel oil kerosene etc.',
 'Percent Coal or coke',
 'Percent Wood',
 'Percent Solar energy',
 'Percent Other fuel',
 'Percent No fuel used',
 'Percent Bottled, tank, or LP gas',
 'Percent Fuel oil, kerosene, etc.',
 'count']

In [140]:
merged_dfs = merged_dfs[['Utility gas',
 'Bottled, tank, or LP gas',
 'Electricity',
 'Fuel oil, kerosene, etc.',
 'Coal or coke',
 'Wood',
 'Solar energy',
 'Other fuel',
 'No fuel used',
 'geoid',
 'incident_year',
 'count']]

In [141]:
merged_dfs['total_used'] = merged_dfs.iloc[:, :9].sum(axis=1)

In [142]:
merged_dfs.head()

,Utility gas,"Bottled, tank, or LP gas",Electricity,"Fuel oil, kerosene, etc.",Coal or coke,Wood,Solar energy,Other fuel,No fuel used,geoid,incident_year,count,total_used
0,1081.0,18.0,37.0,211.0,0.0,0.0,0.0,0.0,0.0,36005000200,2013,2,1347.0
1,1353.0,80.0,163.0,147.0,0.0,0.0,0.0,10.0,0.0,36005000400,2013,5,1753.0
2,1166.0,32.0,122.0,563.0,0.0,0.0,0.0,23.0,39.0,36005001600,2013,11,1945.0
3,381.0,9.0,182.0,259.0,0.0,0.0,0.0,0.0,0.0,36005001900,2013,3,831.0
4,1247.0,39.0,280.0,1352.0,0.0,0.0,0.0,17.0,45.0,36005002000,2013,42,2980.0


In [143]:
pd.set_option('mode.chained_assignment', None)
merged_year = merged_dfs
merged_year['Percent Utility gas']=merged_year['Utility gas']/merged_year['total_used']*100
merged_year['Percent Bottled tank or LP gas']=merged_year['Bottled, tank, or LP gas']/merged_year['total_used']*100
merged_year['Percent Electricity']=merged_year['Electricity']/merged_year['total_used']*100
merged_year['Percent Fuel oil kerosene etc.']=merged_year['Fuel oil, kerosene, etc.']/merged_year['total_used']*100
merged_year['Percent Coal or coke']=merged_year['Coal or coke']/merged_year['total_used']*100
merged_year['Percent Wood']=merged_year['Wood']/merged_year['total_used']*100
merged_year['Percent Solar energy']=merged_year['Solar energy']/merged_year['total_used']*100
merged_year['Percent Other fuel']=merged_year['Other fuel']/merged_year['total_used']*100
merged_year['Percent No fuel used']=merged_year['No fuel used']/merged_year['total_used']*100

merged_year = merged_year[['Percent Utility gas',
 'Percent Bottled tank or LP gas',
 'Percent Electricity',
 'Percent Fuel oil kerosene etc.',
 'Percent Coal or coke',
 'Percent Wood',
 'Percent Solar energy',
 'Percent Other fuel',
 'Percent No fuel used',
'incident_year',
 'count']]


In [144]:
for y in [2013, 2014, 2015, 2016, 2017, 2018]:
    year_data = merged_year[merged_year['incident_year']==y]
    corrMatrix = year_data.corr()['count']
    print(y)
    print (corrMatrix)

2013
Percent Utility gas              -0.286702
Percent Bottled tank or LP gas   -0.051076
Percent Electricity               0.153826
Percent Fuel oil kerosene etc.    0.232183
Percent Coal or coke              0.030090
Percent Wood                     -0.047913
Percent Solar energy             -0.001639
Percent Other fuel                0.162758
Percent No fuel used              0.274786
incident_year                          NaN
count                             1.000000
Name: count, dtype: float64
2014
Percent Utility gas              -0.371835
Percent Bottled tank or LP gas   -0.027913
Percent Electricity               0.263763
Percent Fuel oil kerosene etc.    0.273621
Percent Coal or coke              0.026985
Percent Wood                     -0.053498
Percent Solar energy              0.000634
Percent Other fuel                0.181069
Percent No fuel used              0.305906
incident_year                          NaN
count                             1.000000
Name: count, dty

In [ ]:
# merged_dfs['gas_leaks'] = merged_dfs['count']
# merged_dfs['gas_leaks_per_person'] = merged_dfs['count']/merged_dfs['total_population']
# merged_dfs.drop(columns={'count'}, inplace = True)
# merged_dfs.to_csv(base_path+ 'processed/important_(used_in_app)/nationalities_data.csv')
# nation_all = merged_dfs.groupby(['geoid']).agg({
#      'White':'sum',
#  'Black or African American':'sum',
#  'American Indian and Alaska Native':'sum',
#  'Asian':'sum',
#  'Native Hawaiian and Other Pacific Islander':'sum',
#  'White and Black or African American':'sum',
#  'White and American Indian and Alaska Native':'sum',
#  'White and Asian':'sum',
#  'Black or African American and American Indian and Alaska Native':'sum',
#  'Some other race':'sum',
#  'Hispanic or Latino (of any race)':'sum',
#  'American Indian and Alaska Native:Cherokee tribal grouping':'sum',
#  'American Indian and Alaska Native:Chippewa tribal grouping':'sum',
#  'American Indian and Alaska Native:Navajo tribal grouping':'sum',
#  'American Indian and Alaska Native:Sioux tribal grouping':'sum',
#  'Asian:Asian Indian':'sum',
#  'Asian:Chinese':'sum',
#  'Asian:Filipino':'sum',
#  'Asian:Japanese':'sum',
#  'Asian:Korean':'sum',
#  'Asian:Vietnamese':'sum',
#  'Asian:Other Asian':'sum',
#  'Native Hawaiian and Other Pacific Islander:Native Hawaiian':'sum',
#  'Native Hawaiian and Other Pacific Islander:Guamanian or Chamorro':'sum',
#  'Native Hawaiian and Other Pacific Islander:Samoan':'sum',
#  'Native Hawaiian and Other Pacific Islander:Other Pacific Islander':'sum',
#  'Hispanic or Latino (of any race):Mexican':'sum',
#  'Hispanic or Latino (of any race):Puerto Rican':'sum',
#  'Hispanic or Latino (of any race):Cuban':'sum',
#  'Hispanic or Latino (of any race):Other Hispanic or Latino':'sum',
#  'total_population':'sum',
#  'gas_leaks':'sum'
# }).reset_index()
# merged_dfs.to_csv(base_path+ 'processed/important_(used_in_app)/nationalities_data_all.csv')